In [1]:
import numpy as np
import pandas as pd

from itertools import combinations

In [5]:
df = pd.read_csv('labeled.csv', index_col=0)
df.head()

,City,State,Population,Percent employed,Occupation (MBSA),Occupation (S),Occupation (SO),Occupation (RCM),Occupation (PT),Median household income,Homeownership rate,Median home price,Median rent,KMeans,Hierarchical
0,Acres Green,Colorado,2709.0,72.7,61.9,11.9,12.5,8.5,5.2,115536.0,77.4,540000.0,2663.0,0,3
1,Aetna Estates,Colorado,1177.0,72.4,12.5,27.3,31.8,17.0,11.4,63887.0,34.3,58200.0,1490.0,8,4
2,Akron,Colorado,1681.0,55.4,40.5,19.3,21.5,13.2,5.5,59470.0,70.4,186500.0,1070.0,5,0
3,Alamosa,Colorado,9847.0,61.8,34.7,20.7,26.5,6.6,11.4,46969.0,43.8,200200.0,881.0,8,4
4,Alamosa East,Colorado,1404.0,68.4,34.9,29.7,17.9,9.1,8.4,44741.0,68.5,219100.0,1100.0,3,4


In [6]:
df.shape

(1967, 15)

In [7]:
vectors = df.drop(columns=['City', 'State', 'KMeans', 'Hierarchical']).to_numpy()
vectors.shape

(1967, 11)

In [8]:
norms = np.linalg.norm(vectors, axis=1)
norms.shape

(1967,)

In [9]:
combos = combinations(range(0, 1966), 2)

In [10]:
cos_matrix = np.empty((1967, 1967))
for combo in combos:
    i = combo[0]
    j = combo[1]
    numerator = np.dot(vectors[i], vectors[j])
    denominator = norms[i] * norms[j]
    cos_sim = numerator / denominator
    cos_matrix[i,j] = cos_sim
    cos_matrix[j,i] = cos_sim  

NOTE TO SELF: I should run this operation three times to get the three most similar cities. Example workflow:
- Create array of size (1967, 3)
- Run loop once
- Get argmax
- Save indices to first column of array
- Use indices to set those intersections to 0
- Rerun loop 2 more times to fill out initial array
- Use array to pull up similar cities in streamlit app

In [ ]:
top_3 = np.empty((1967, 3))
for i in range(3):
    most_similar = np.argmax(cos_matrix, axis=1)
    top_3[:,i] = most_similar
    #if this is the last pass, don't bother setting more values to 0
    if i >= 2:
        break
    #set all matches to 0 for the next pass through the loop
    for j in range(len(most_similar)):
        cos_matrix[j, most_similar[j]] = 0

#check out the result
top_3[:5]

array([[1182.,  463.,  334.],
       [1635., 1933.,  639.],
       [ 832.,  430., 1082.],
       [  92., 1547., 1794.],
       [  58.,  225.,  424.]])

In [27]:
#change to int dtype so it can be used to index the dataframe
top_3.astype(int, copy=False)
#save array so it can be pulled in by streamlit app
np.save('top_3.npy', top_3)